In [44]:
import pandas as pd
import json
import os
import requests
from dotenv import load_dotenv
import openai

load_dotenv()

True

In [45]:
BING_KEY = os.environ["BING_SEARCH_V7_SUBSCRIPTION_KEY"]
BING_API_ENDPOINT = os.environ["BING_SEARCH_V7_ENDPOINT"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [19]:
def get_news(query="", count=100):
    headers = {"Ocp-Apim-Subscription-Key": BING_KEY}
    params = {"mkt": "en-US", "q": query, "count": count, "sortBy": "Date"}
    response = requests.get(BING_API_ENDPOINT, headers=headers, params=params)
    response.raise_for_status()
    news_articles = response.json()
    return news_articles

In [26]:
location = "Bogota, Colombia"
event = "natural disaster"

query = f"{location} {event}"
news_articles = get_news(query=query)
print(f"fetched {len(news_articles['value'])} news articles for {query}")

fetched 27 news articles for Bogota, Colombia natural disaster


In [40]:
json.dump(news_articles, open("data/bogota-natural-disaster-news.json", "w"), indent=4)

In [64]:
parsed_articles = [{
    "title": a["name"],
    "content": a["description"],
    "timestamp": a["datePublished"]}
    for a in news_articles['value']
]

#### Try analysing news with GPT

In [65]:
from openai import OpenAI

openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [67]:
SYSTEM_PROMPT = """
You're an analyst for Outokumpu, a stainless steel manufacturer. You're tasked with monitoring news articles for mentions of natural disasters that could impact the company's operations.\
You have an important raw material supplier in Bogota, Colombia. You monitor news articles for mentions of natural disasters in Bogota, Colombia.\
If there is a natural disaster that could impact the company's operations, you need to mention it in your daily report.\
If the news don't contain important information, you MUST NOT mention it in your daily report.\
Your report is of the form:
Potential natural disasters in Bogota, Colombia: <boolean>
If <boolean> is true, output the following:
RELEVANT NEWS ARTICLES: [[{title}, {content}], ...]
"""
SYSTEM_PROMPT

"\nYou're an analyst for Outokumpu, a stainless steel manufacturer. You're tasked with monitoring news articles for mentions of natural disasters that could impact the company's operations.You have an important raw material supplier in Bogota, Colombia. You monitor news articles for mentions of natural disasters in Bogota, Colombia.If there is a natural disaster that could impact the company's operations, you need to mention it in your daily report.If the news don't contain important information, you MUST NOT mention it in your daily report.Your report is of the form:\nPotential natural disasters in Bogota, Colombia: <boolean>\nIf <boolean> is true, output the following:\nRELEVANT NEWS ARTICLES: [[{title}, {content}], ...]\n"

In [89]:
QUERY_PROMPT = ""
for article in parsed_articles[:3]:
    QUERY_PROMPT += f"Title: {article['title']}\nContent: {article['content']}\n\n"

len(QUERY_PROMPT)

865

In [75]:
chat_completion = openai_client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": SYSTEM_PROMPT,
        },
        {
            "role": "user",
            "content": QUERY_PROMPT,
        }
    ],
    model="gpt-3.5-turbo",
)

In [86]:
chat_completion.choices[0].message.content

'Potential natural disasters in Bogota, Colombia: False'

In [90]:
fake_disaster_article = {
    "title": "Huge earthquake in Bogota, Colombia",
    "content": "A huge earthquake hit Bogota, Colombia, causing a lot of damage. The earthquake was 8.0 on the Richter scale."
}

QUERY_PROMPT_WITH_BAD_EVENT = QUERY_PROMPT + f"Title: {fake_disaster_article['title']}\nContent: {fake_disaster_article['content']}\n\n"
QUERY_PROMPT_WITH_BAD_EVENT

"Title: 6 questions about travel to Colombia with Bay Village writer and Lonely Planet contributor Laura Watilo Blake\nContent: Laura Watilo Blake took her first trip to Colombia in 2010. She returned six years later to adopt her daughter. She’s has been back a dozen times since, most recently to research for Lonely Planet Colombia,\n\nTitle: Deforestation in Colombia Down 70 Percent So Far This Year\nContent: Deforestation in the Colombian Amazon is down 70 percent, year on year, through the first nine months of 2023, the government estimates.\n\nTitle: Colombia Amazon deforestation seen down 70% through September -minister\nContent: Deforestation in Colombia's Amazon is forecast to have fallen 70% in the first nine months of 2023 from the year-ago period, Environment Minister Susana Muhamad said on Tuesday, though she warned drought could reverse the trend.\n\nTitle: Huge earthquake in Bogota, Colombia\nContent: A huge earthquake hit Bogota, Colombia, causing a lot of damage. The ear

In [87]:
chat_completion_2 = openai_client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": SYSTEM_PROMPT,
        },
        {
            "role": "user",
            "content": QUERY_PROMPT,
        }
    ],
    model="gpt-3.5-turbo",
)

In [91]:
chat_completion_2.choices[0].message.content

'Potential natural disasters in Bogota, Colombia: True\n\nRELEVANT NEWS ARTICLES: [["Huge earthquake in Bogota, Colombia", "A huge earthquake hit Bogota, Colombia, causing a lot of damage. The earthquake was 8.0 on the Richter scale."]]'